<a href="https://colab.research.google.com/github/smmurdock/Solve-an-Optimization-Problem/blob/main/D605_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# D605 Task 3 - Identification of the Objective Function and Constraints

## A. Optimization Problem Program

> Using the mathematical representations that you developed in Task 2, write a program in either Python or R to solve the optimization problem computationally.

In [1]:
# !pip install pulp

In [2]:
import pulp
import sys

In [3]:
# Data definitions as described from "Amazon Distribution.docx"
# --- Define the sets of locations ---
hubs = ['CVG', 'AFW']
focus_cities = ['Leipzig', 'Hyderabad', 'San Bernardino']
fulfillment_centers = [
    'Paris', 'Cologne', 'Hanover', 'Bangalore',
    'Coimbatore', 'Delhi', 'Mumbai', 'Cagliari',
    'Catania', 'Milan', 'Rome', 'Katowice',
    'Barcelona', 'Madrid', 'Castle Donington', 'London',
    'Mobile', 'Anchorage', 'Fairbanks', 'Phoenix',
    'Los Angeles', 'Ontario', 'Riverside', 'Sacramento',
    'San Francisco', 'Stockton', 'Denver', 'Hartford',
    'Miami', 'Lakeland', 'Tampa', 'Atlanta',
    'Honolulu', 'Kahului/Maui', 'Kona', 'Chicago',
    'Rockford', 'Fort Wayne', 'South Bend', 'Des Moines',
    'Wichita', 'New Orleans', 'Baltimore', 'Minneapolis',
    'Kansas City', 'St. Louis', 'Omaha', 'Manchester',
    'Albuquerque', 'New York', 'Charlotte', 'Toledo',
    'Wilmington', 'Portland', 'Allentown', 'Pittsburgh',
    'San Juan', 'Nashville', 'Austin', 'Dallas',
    'Houston', 'San Antonio', 'Richmond', 'Seattle/Tacoma',
    'Spokane'
]

# --- Define capacities for hubs and focus cities ---
hub_capacities_per_month = {
    'CVG': 95_650,
    'AFW': 44_350
}

focus_city_capacities_per_month = {
    'Leipzig': 85_000,
    'Hyderabad': 19_000,
    'San Bernardino': 36_000
}

# --- Define demand for each fulfillment center ---
center_demands_per_month = {
    'Paris': 6500, 'Cologne': 640, 'Hanover': 180, 'Bangalore': 9100,
    'Coimbatore': 570, 'Delhi': 19000, 'Mumbai': 14800, 'Cagliari': 90,
    'Catania': 185, 'Milan': 800, 'Rome': 1700, 'Katowice': 170,
    'Barcelona': 2800, 'Madrid': 3700, 'Castle Donington': 30, 'London': 6700,
    'Mobile': 190, 'Anchorage': 175, 'Fairbanks': 38, 'Phoenix': 2400,
    'Los Angeles': 7200, 'Ontario': 100, 'Riverside': 1200, 'Sacramento': 1100,
    'San Francisco': 1900, 'Stockton': 240, 'Denver': 1500, 'Hartford': 540,
    'Miami': 3400, 'Lakeland': 185, 'Tampa': 1600, 'Atlanta': 3000,
    'Honolulu': 500, 'Kahului/Maui': 16, 'Kona': 63, 'Chicago': 5100,
    'Rockford': 172, 'Fort Wayne': 200, 'South Bend': 173, 'Des Moines': 300,
    'Wichita': 290, 'New Orleans': 550, 'Baltimore': 1300, 'Minneapolis': 1700,
    'Kansas City': 975, 'St. Louis': 1200, 'Omaha': 480, 'Manchester': 100,
    'Albuquerque': 450, 'New York': 11200, 'Charlotte': 900, 'Toledo': 290,
    'Wilmington': 150, 'Portland': 1200, 'Allentown': 420, 'Pittsburgh': 1000,
    'San Juan': 1100, 'Nashville': 650, 'Austin': 975, 'Dallas': 3300,
    'Houston': 3300, 'San Antonio': 1100, 'Richmond': 600, 'Seattle/Tacoma': 2000,
    'Spokane': 260
}

# --- Define transportation costs ---
# Keys: tuples (source, destination)
costs = {
    ('CVG', 'Leipzig'): 1.5, ('CVG', 'San Bernardino'): 0.5,
    ('AFW', 'San Bernardino'): 0.5,
    ('CVG', 'Paris'): 1.6, ('CVG', 'Cologne'): 1.5, ('CVG', 'Hanover'): 1.5,
    ('CVG', 'Cagliari'): 1.5, ('CVG', 'Catania'): 1.5, ('CVG', 'Milan'): 1.5,
    ('CVG', 'Rome'): 1.5, ('CVG', 'Katowice'): 1.4, ('CVG', 'Barcelona'): 1.5,
    ('CVG', 'Madrid'): 1.6, ('CVG', 'Castle Donington'): 1.4, ('CVG', 'London'): 1.6,
    ('CVG', 'Mobile'): 0.5, ('CVG', 'Anchorage'): 1.3, ('CVG', 'Fairbanks'): 1.4,
    ('CVG', 'Phoenix'): 0.5, ('CVG', 'Los Angeles'): 0.5, ('CVG', 'Ontario'): 0.5,
    ('CVG', 'Riverside'): 0.5, ('CVG', 'Sacramento'): 0.5, ('CVG', 'San Francisco'): 0.5,
    ('CVG', 'Stockton'): 0.5, ('CVG', 'Denver'): 0.5, ('CVG', 'Hartford'): 0.5,
    ('CVG', 'Miami'): 0.5, ('CVG', 'Lakeland'): 0.5, ('CVG', 'Tampa'): 0.5,
    ('CVG', 'Atlanta'): 0.5, ('CVG', 'Chicago'): 0.5, ('CVG', 'Rockford'): 0.5,
    ('CVG', 'Fort Wayne'): 0.5, ('CVG', 'South Bend'): 0.5, ('CVG', 'Des Moines'): 0.5,
    ('CVG', 'Wichita'): 0.5, ('CVG', 'New Orleans'): 0.5, ('CVG', 'Baltimore'): 0.5,
    ('CVG', 'Minneapolis'): 0.5, ('CVG', 'Kansas City'): 0.5, ('CVG', 'St. Louis'): 0.5,
    ('CVG', 'Omaha'): 0.5, ('CVG', 'Manchester'): 0.5, ('CVG', 'Albuquerque'): 0.5,
    ('CVG', 'New York'): 0.5, ('CVG', 'Charlotte'): 0.5, ('CVG', 'Toledo'): 0.5,
    ('CVG', 'Wilmington'): 0.5, ('CVG', 'Portland'): 0.5, ('CVG', 'Allentown'): 0.5,
    ('CVG', 'Pittsburgh'): 0.5, ('CVG', 'San Juan'): 0.5, ('CVG', 'Nashville'): 0.5,
    ('CVG', 'Austin'): 0.5, ('CVG', 'Dallas'): 0.5, ('CVG', 'Houston'): 0.5,
    ('CVG', 'San Antonio'): 0.5, ('CVG', 'Richmond'): 0.5, ('CVG', 'Seattle/Tacoma'): 0.5,
    ('CVG', 'Spokane'): 0.5,
    ('AFW', 'Mobile'): 0.5, ('AFW', 'Anchorage'): 1.0, ('AFW', 'Fairbanks'): 1.0,
    ('AFW', 'Phoenix'): 0.5, ('AFW', 'Los Angeles'): 0.5, ('AFW', 'Ontario'): 0.5,
    ('AFW', 'Riverside'): 0.5, ('AFW', 'Sacramento'): 0.5, ('AFW', 'San Francisco'): 0.5,
    ('AFW', 'Stockton'): 0.5, ('AFW', 'Denver'): 0.5, ('AFW', 'Hartford'): 0.5,
    ('AFW', 'Miami'): 0.5, ('AFW', 'Lakeland'): 0.5, ('AFW', 'Tampa'): 0.5,
    ('AFW', 'Atlanta'): 0.5, ('AFW', 'Honolulu'): 0.5, ('AFW', 'Kahului/Maui'): 0.5,
    ('AFW', 'Kona'): 0.5, ('AFW', 'Chicago'): 0.5, ('AFW', 'Rockford'): 0.5,
    ('AFW', 'Fort Wayne'): 0.5, ('AFW', 'South Bend'): 0.5, ('AFW', 'Des Moines'): 0.5,
    ('AFW', 'Wichita'): 0.5, ('AFW', 'New Orleans'): 0.5, ('AFW', 'Baltimore'): 0.5,
    ('AFW', 'Minneapolis'): 0.5, ('AFW', 'Kansas City'): 0.5, ('AFW', 'St. Louis'): 0.5,
    ('AFW', 'Omaha'): 0.5, ('AFW', 'Manchester'): 0.5, ('AFW', 'Albuquerque'): 0.5,
    ('AFW', 'New York'): 0.5, ('AFW', 'Charlotte'): 0.5, ('AFW', 'Toledo'): 0.5,
    ('AFW', 'Wilmington'): 0.5, ('AFW', 'Portland'): 0.5, ('AFW', 'Allentown'): 0.5,
    ('AFW', 'Pittsburgh'): 0.5, ('AFW', 'San Juan'): 0.5, ('AFW', 'Nashville'): 0.5,
    ('AFW', 'Austin'): 0.25, ('AFW', 'Houston'): 0.25, ('AFW', 'San Antonio'): 0.25,
    ('AFW', 'Richmond'): 0.5, ('AFW', 'Seattle/Tacoma'): 0.5, ('AFW', 'Spokane'): 0.5,
    ('Leipzig', 'Hyderabad'): 1.6, ('Leipzig', 'Paris'): 0.5, ('Leipzig', 'Cologne'): 0.5,
    ('Leipzig', 'Hanover'): 0.5, ('Leipzig', 'Bangalore'): 1.5, ('Leipzig', 'Coimbatore'): 1.5,
    ('Leipzig', 'Delhi'): 1.5, ('Leipzig', 'Mumbai'): 1.5, ('Leipzig', 'Cagliari'): 0.5,
    ('Leipzig', 'Catania'): 0.5, ('Leipzig', 'Milan'): 0.5, ('Leipzig', 'Rome'): 0.5,
    ('Leipzig', 'Katowice'): 0.5, ('Leipzig', 'Barcelona'): 0.5, ('Leipzig', 'Madrid'): 0.5,
    ('Leipzig', 'Castle Donington'): 0.5, ('Leipzig', 'London'): 0.75, ('Leipzig', 'Hartford'): 1.5,
    ('Leipzig', 'Baltimore'): 1.5, ('Leipzig', 'Manchester'): 1.5, ('Leipzig', 'New York'): 1.6,
    ('Leipzig', 'Allentown'): 1.5,
    ('Hyderabad', 'Paris'): 1.1, ('Hyderabad', 'Cologne'): 1.0, ('Hyderabad', 'Hanover'): 1.0,
    ('Hyderabad', 'Bangalore'): 0.5, ('Hyderabad', 'Coimbatore'): 0.5, ('Hyderabad', 'Delhi'): 0.5,
    ('Hyderabad', 'Mumbai'): 0.5, ('Hyderabad', 'Cagliari'): 1.0, ('Hyderabad', 'Catania'): 1.0,
    ('Hyderabad', 'Milan'): 1.0, ('Hyderabad', 'Rome'): 1.1, ('Hyderabad', 'Katowice'): 1.0,
    ('Hyderabad', 'Barcelona'): 1.0, ('Hyderabad', 'Madrid'): 1.1, ('Hyderabad', 'London'): 1.1,
    ('San Bernardino', 'Mobile'): 0.5, ('San Bernardino', 'Anchorage'): 0.7, ('San Bernardino', 'Fairbanks'): 0.7,
    ('San Bernardino', 'Phoenix'): 0.5, ('San Bernardino', 'Sacramento'): 0.5, ('San Bernardino', 'San Francisco'): 0.5,
    ('San Bernardino', 'Stockton'): 0.5, ('San Bernardino', 'Denver'): 0.5, ('San Bernardino', 'Hartford'): 0.5,
    ('San Bernardino', 'Miami'): 0.7, ('San Bernardino', 'Lakeland'): 0.7, ('San Bernardino', 'Tampa'): 0.7,
    ('San Bernardino', 'Atlanta'): 0.6, ('San Bernardino', 'Honolulu'): 0.5, ('San Bernardino', 'Kahului/Maui'): 0.5,
    ('San Bernardino', 'Kona'): 0.5, ('San Bernardino', 'Chicago'): 0.5, ('San Bernardino', 'Rockford'): 0.5,
    ('San Bernardino', 'Fort Wayne'): 0.5, ('San Bernardino', 'South Bend'): 0.5, ('San Bernardino', 'Des Moines'): 0.5,
    ('San Bernardino', 'Wichita'): 0.5, ('San Bernardino', 'New Orleans'): 0.5, ('San Bernardino', 'Baltimore'): 0.7,
    ('San Bernardino', 'Minneapolis'): 0.5, ('San Bernardino', 'Kansas City'): 0.5, ('San Bernardino', 'St. Louis'): 0.5,
    ('San Bernardino', 'Omaha'): 0.5, ('San Bernardino', 'Manchester'): 0.7, ('San Bernardino', 'Albuquerque'): 0.5,
    ('San Bernardino', 'New York'): 0.7, ('San Bernardino', 'Charlotte'): 0.7, ('San Bernardino', 'Toledo'): 0.5,
    ('San Bernardino', 'Wilmington'): 0.7, ('San Bernardino', 'Portland'): 0.5, ('San Bernardino', 'Allentown'): 0.7,
    ('San Bernardino', 'Pittsburgh'): 0.6, ('San Bernardino', 'San Juan'): 1.0, ('San Bernardino', 'Nashville'): 0.5,
    ('San Bernardino', 'Austin'): 0.5, ('San Bernardino', 'Dallas'): 0.5, ('San Bernardino', 'Houston'): 0.5,
    ('San Bernardino', 'San Antonio'): 0.5, ('San Bernardino', 'Richmond'): 0.7, ('San Bernardino', 'Seattle/Tacoma'): 0.5,
    ('San Bernardino', 'Spokane'): 0.5,
}

In [4]:
# Problem initialization
# Create the linear programming problem
# Resource: (PuLP Documentation - A Transportation Problem)
prob = pulp.LpProblem("Amazon_Air_Cargo_Optimization", pulp.LpMinimize)

In [5]:
# Decision Variables
# x_ij, y_ik, z_jk

# --- x_ij: hub i to focus city j ---
# Create tuples of valid routes for x
routes_x = [(h, fc) for h in hubs for fc in focus_cities if (h, fc) in costs]
x = pulp.LpVariable.dicts("Shipment_Hub_to_Focus_City", routes_x, lowBound=0, cat='Continuous')

# --- y_ik: hub i to fulfillment center k ---
# Create tuples of valid routes for y
routes_y = [(h, c) for h in hubs for c in fulfillment_centers if (h, c) in costs]
y = pulp.LpVariable.dicts("Shipment_Hub_to_Fulfillment_Center", routes_y, lowBound=0, cat='Continuous')

# --- z_jk: focus city j to fulfillment center k ---
# Create tuples of valid routes for z
routes_z = [(fc, c) for fc in focus_cities for c in fulfillment_centers if (fc, c) in costs]
z = pulp.LpVariable.dicts("Shipment_Focus_to_Fulfillment_Center", routes_z, lowBound=0, cat='Continuous')

In [6]:
# Objective function
prob += (
    pulp.lpSum([x[route] * costs[route] for route in routes_x]) +
    pulp.lpSum([y[route] * costs[route] for route in routes_y]) +
    pulp.lpSum([z[route] * costs[route] for route in routes_z]),
    "Total_Transportation_Cost",
)

In [7]:
# Contraints
# --- 1. Hub Capacity Constraints ---
for h in hubs:
    prob += (
        pulp.lpSum([x[(h, fc)] for fc in focus_cities if (h, fc) in routes_x]) +
        pulp.lpSum([y[(h, c)] for c in fulfillment_centers if (h, c) in routes_y])
        <= hub_capacities_per_month[h],
        f"Hub_Capacity_{h}",
    )

# --- 2. Focus City Capacity Constraints (Inbound) ---
for fc in focus_cities:
    prob += (
        pulp.lpSum([x[(h, fc)] for h in hubs if (h, fc) in routes_x])
        <= focus_city_capacities_per_month[fc],
        f"Focus_City_Capacity_Inbound_{fc}",
    )

# --- 3. Flow Balance at Focus Cities (Outbound must equal inbound)
for fc in focus_cities:
    prob += (
        pulp.lpSum([z[(fc, c)] for c in fulfillment_centers if (fc, c) in routes_z])
        == pulp.lpSum([x[(h, fc)] for h in hubs if (h, fc) in routes_x]),
        f"Flow_Balance_{fc}",
    )

# --- 4. Fulfillment Center Demand Constraints ---
for c in fulfillment_centers:
    prob += (
        pulp.lpSum([y[(h, c)] for h in hubs if (h, c) in routes_y]) +
        pulp.lpSum([z[(fc, c)] for fc in focus_cities if (fc, c) in routes_z])
        == center_demands_per_month[c],
        f"Demand_Satisfaction_{c.replace(' ', '_').replace('/', '_')}",
    )

### A1. Demonstrate Solver Solution

> Demonstrate that the solver provided a solution to the optimization problem.

In [10]:
# Solve the Problem
# --- The problem is solved using PuLP's default solver (CBC) ---
prob.solve()

# --- Display the Results ---
print("=" * 80)
print(f"Optimization Solution")
print("=" * 80)
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total Cost: ${pulp.value(prob.objective):,.2f}")


print("=" * 80)
print("\nOptimal Shipping Plan (in tons)")

print("-" * 80)
print("\nHub to Focus City Shipments")
for (h, fc) in routes_x:
    if x[(h, fc)].varValue > 0:
        print(f"-  {h} -> {fc}: ${x[(h, fc)].varValue:,.2f}")

print("-" * 80)
print("\nHub to Center Shipments")
for (h, c) in routes_y:
    if y[(h, c)].varValue > 0:
        print(f"-  {h} -> {c}: ${y[(h, c)].varValue:,.2f}")

print("-" * 80)
print("\nFocus City to Center Shipments")
for (fc, c) in routes_z:
    if z[(fc, c)].varValue > 0:
        print(f"-  {fc} -> {c}: ${z[(fc, c)].varValue:,.2f}")

Optimization Solution
Status: Optimal
Total Cost: $199,476.25

Optimal Shipping Plan (in tons)
--------------------------------------------------------------------------------

Hub to Focus City Shipments
-  CVG -> Leipzig: $43,470.00
--------------------------------------------------------------------------------

Hub to Center Shipments
-  CVG -> Paris: $6,500.00
-  CVG -> Cologne: $640.00
-  CVG -> Hanover: $180.00
-  CVG -> Cagliari: $90.00
-  CVG -> Catania: $185.00
-  CVG -> Milan: $800.00
-  CVG -> Rome: $1,700.00
-  CVG -> Katowice: $170.00
-  CVG -> Barcelona: $2,800.00
-  CVG -> Madrid: $3,700.00
-  CVG -> Castle Donington: $30.00
-  CVG -> London: $6,700.00
-  CVG -> Mobile: $190.00
-  CVG -> Los Angeles: $7,200.00
-  CVG -> Ontario: $100.00
-  CVG -> Riverside: $1,200.00
-  CVG -> Sacramento: $1,100.00
-  CVG -> San Francisco: $825.00
-  CVG -> Denver: $1,500.00
-  CVG -> Wichita: $290.00
-  CVG -> Baltimore: $1,300.00
-  CVG -> Omaha: $480.00
-  CVG -> New York: $11,200.00

## B. Output Analysis

> Analyze the output of your model to demonstrate that the solution satisfies the requirements of the problem.

### B1. Satisfying Optimization Problem Constraints

> Demonstrate that the constraints of the optimization problem are satisfied.

### B2. Decision Variables, Constraints, and Objective Function

> Demonstrate that the solution includes decision variables, constraints, and the objective function.

### B3. Expected Output Solution

> Explain why the solution that your code outputs matches your expected output.

## C. Reflection

> Provide a reflection on how the development of your approach matched your expectations of the process.

## D. Sources

> Acknowledge sources, using in-text citations and references, for content that is quoted, paraphrased, or summarized.



* [PuLP Documentation - A Transportation Problem](https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html)
* WGU Course Materials